In [1]:
import pandas as pd
from groq import Groq
import itertools
import tqdm
import time
import os
import random
import re
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser

In [2]:
data = []
for file in list(filter(lambda x: x.startswith('p-'), os.listdir('../md_files/'))):
    with open('../md_files/' + file, 'r', encoding='utf-8') as file:
        content = file.read()
        chunks = list(map(lambda x: re.sub(r"# ACÓRDÃO.*\n", "", x).strip(), filter(lambda x: x not in ['RELATÓRIO', 'VOTO'], re.split(r"# (RELATÓRIO|VOTO)", content))))
        data.append({
            "acordao": chunks[0],
            "relatorio": chunks[1],
            "voto": chunks[2]
        })
df = pd.DataFrame(data)
df

,acordao,relatorio,voto
0,"Vistos, relatados e discutidos os autos do pro...",Tratam os autos da aposentadoria da servidora ...,Trata-se de apreciação da legalidade e registr...
1,"Vistos, relatados e discutidos os autos do pro...",1. Tratam os autos da Prestação de Contas do F...,A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...
2,"Vistos, relatados e discutidos os autos do pro...",1. Tratam os autos da Prestação de Contas da S...,A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...
3,"Vistos, relatados e discutidos os autos do pro...","1. Trata-se os autos de Denúncia, oriunda do o...",A EXMA. SENHORA CONSELHEIRA NALUH MARIA LIMA G...
4,"Vistos, relatados e discutidos os autos do pro...",1. Trata o presente processo da aposentadoria ...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...
...,...,...,...
145,"Vistos, relatados e discutidos os autos do pro...",1. Tratam os autos da Prestação de Contas do G...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...
146,"Vistos, relatados e discutidos os autos do pro...",1. Tratam os autos de Denúncia protocolada na ...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...
147,"Vistos, relatados e discutidos os autos do pro...",Tratam os autos da aposentadoria da servidora ...,Trata-se de apreciação da legalidade e registr...
148,"Vistos, relatados e discutidos os autos do pro...",1. Trata o presente processo da aposentadoria ...,A EXMA. SENHORA CONSELHEIRA-SUBSTITUTA MARIA D...


### Examples Definition

In [3]:
example1 = """
Relatório:
Trata o presente processo de aposentadoria voluntária por tempo de contribuição da servidora pública estadual Eutália Paz da Silva, no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal do Estado da Secretaria de Estado de Saúde, pelas regras previstas no artigo 3º da Emenda Constitucional nº 47, de 05 de julho de 2005.

Da análise dos autos e de acordo com o Relatório Técnico (fls. 94-95), apurou-se que:

1. A servidora conta com a idade de 57 (cinquenta e sete) anos e 37 (trinta e sete) anos e 195 (cento e noventa e cinco) dias de contribuição, para fins de aposentadoria;
2. A aposentadoria foi concedida no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal da Secretaria de Estado de Saúde, através da Portaria nº 98 de 10/02/2021;
3. Os proventos mensais iniciais foram calculados no valor de R$1.942,42 (mil novecentos e quarenta e dois reais e quarenta e dois centavos), conforme Ato de Fixação dos Proventos (fls. 79);
4. Atualmente a servidora faz jus aos proventos no valor de R$ 2.341,74 (dois mil trezentos e quarenta e um reais e setenta e quatro centavos), de acordo com a Ficha Financeira atualizada (fls. 96-157).

O Ministério Público de Contas se manifestou (fls. 162-163) em pronunciamento da lavra da Procuradora Anna Helena de Azevedo Lima, tendo se manifestado favorável pela legalidade e registro da matéria.

É o RELATÓRIO.

Voto:
O EXMO. SENHOR CONSELHEIRO VALMIR GOMES RIBEIRO (Relator):

Visto e analisado o presente processo, e ainda consubstanciado na Análise Técnica (fls. 94-95), bem como no Parecer do Ministério Público de Contas (fls. 162-163), concluo votando:

1. pela legalidade e registro da aposentadoria voluntária por tempo de contribuição da servidora pública estadual Eutália Paz da Silva, CPF nº 112.709.342-87, matrícula nº 91456-1, no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal do Estado da Secretaria de Estado de Saúde, concedida através da Portaria nº 98 de 10/02/2021, com fundamento no artigo 3º da Emenda Constitucional nº 47, de 05 de julho de 2005;
2. pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada. Se impresso, para conferência acesse o site http://www.tceac.tc.br/conferencia e informe o código 01335634.
3. após, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, na 127ª Sessão Ordinária Virtual, ACORDAM os Membros da 2ª Câmara do Tribunal de Contas do Estado do Acre, à unanimidade, nos termos do voto do Conselheiro-Relator, com fulcro no artigo 61, inciso III, da Constituição Estadual:

1. pela legalidade e registro da aposentadoria voluntária por tempo de contribuição da servidora pública estadual Eutália Paz da Silva, CPF nº 112.709.342-87, matrícula nº 91456-1, com fundamento no artigo 3º da Emenda Constitucional nº 47, de 05 de julho de 2005, no cargo de Auxiliar Operacional de Serviços Diversos, Grupo I, Referência 8, do Quadro de Pessoal do Estado da Secretaria de Estado de Saúde, concedida através da Portaria nº 98 de 10/02/2021;
2. pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

In [4]:
example2 = """
Relatório:
Trata o presente processo de aposentadoria voluntária integral especial por tempo de contribuição da servidora pública estadual Marizete Correia de Andrade Carneiro, no cargo de Professora P2, 30 horas, Classe II, Referência J, do Quadro de Pessoal do Estado da Secretaria de Estado de Educação, Cultura e Esportes, pelas regras previstas no artigo 5º, §1º, da Emenda Constitucional do Estado nº 52, de 02 de dezembro de 2019 e §2º, inciso I, do mesmo artigo.

Da análise dos autos e de acordo com o Relatório Técnico (fls. 162-164), apurou-se que:

1. A servidora conta com a idade de 52 (cinquenta e dois) anos e 34 (trinta e quatro) anos e 154 (cento e cinquenta e quatro) dias de contribuição, para fins de aposentadoria;
2. A aposentadoria foi concedida no cargo de Professora P2, 30 horas, Classe II, Referência 6, do Quadro de Pessoal da Secretaria de Estado de Educação, Cultura e Esportes, através da Portaria nº 613 de 20/09/2022;
3. Os proventos mensais iniciais foram calculados no valor de R$ 5.148,03 (cinco mil cento e quarenta e oito reais e três centavos), conforme Ato de Fixação dos Proventos (fl. 138);
4. Observa-se, no entanto, que o enquadramento da servidora no ato de aposentadoria foi equivocado, uma vez que, observando-se o art. 29 da LCE nº 67/99 com redação dada pelo art. 29, §8º, da LCE nº 330/17, a servidora deveria ter sido aposentada na Referência “J”, e seus proventos mensais, com a devida correção, ter sido fixado em R$ 6.215,78 (seis mil duzentos e quinze reais e setenta e oito centavos);
5. atualmente a servidora faz jus aos proventos no valor de R$ 7.017,77 (sete mil e dezessete reais e setenta e sete centavos), de acordo com a Ficha Financeira atualizada (fls. 165-216).

O Ministério Público de Contas se manifestou (fls. 221) em pronunciamento da lavra do Procurador Anna Helena de Azevedo Lima, tendo se manifestado favorável pela legalidade e registro da matéria, sem prejuízo da cientificação da beneficiária a respeito e assinação de prazo para a correção da origem.

É o RELATÓRIO.

Voto:
O EXMO. SENHOR CONSELHEIRO VALMIR GOMES RIBEIRO (Relator):

Visto e analisado o presente processo, e ainda consubstanciado na Análise Técnica (fls. 162-164), bem como no Parecer do Ministério Público de Contas (fls. 221), concluo votando:

1. pela legalidade e registro da aposentadoria voluntária integral especial por tempo de contribuição da servidora pública estadual Marizete Correia de Andrade Carneiro, CPF nº 308.350.892-15, matrícula nº 210366-1, no cargo de Professora P2, 30 horas, Classe II, Referência J, do Quadro de Pessoal do Estado da Secretaria de Estado de Educação, Cultura e Esportes, concedida através da Portaria nº 613 de 20/09/2022, com fundamento no artigo 5º, §1º, da Emenda Constitucional do Estado nº 52, de 02 de dezembro de 2019 e §2º, inciso I, do mesmo artigo;
2. pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada.
3. após, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, na 127ª Sessão Ordinária Virtual, ACORDAM os Membros da 2ª Câmara do Tribunal de Contas do Estado do Acre, à unanimidade, nos termos do voto do Conselheiro-Relator, com fulcro no artigo 61, inciso III, da Constituição Estadual: 1) pela legalidade e registro da aposentadoria voluntária integral especial por tempo de contribuição da servidora pública estadual Marizete Correia de Andrade Carneiro, CPF nº 308.350.892-15, matrícula nº 210366-1, com fundamento no artigo 5º, §1º, da Emenda Constitucional do Estado nº 52, de 02 de dezembro de 2019 e §2º, inciso I, do mesmo artigo, no cargo de Professora P2, 30 horas, Classe II, Referência J, do Quadro de Pessoal do Estado da Secretaria de Estado de Educação, Cultura e Esportes, concedida através da Portaria nº 613 de 20/09/2022; 2) pelo encaminhamento de cópia do processo ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA para as providências cabíveis e para que promova a ciência da servidora interessada. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

In [5]:
example3 = """
Relatório:
1. Tratam os autos da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula nº 266027-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J.
2. A servidora foi admitida na Secretaria Estadual de Educação por concurso público em 1º de dezembro de 1993 para ocupar o cargo de Professora PE5, 40 horas. Conforme fichas financeiras da servidora esta recebeu todas as promoções e progressões da carreira do magistério, nos termos da Lei complementar n. 67/1999 e, nos termos desta Lei foi enquadrada como professora P2, 25 horas, a partir de 16 de junho 1999. Posteriormente, por meio da LC n. 91, de 14.02.2001 que alterou a LC n. 67/99 a servidora foi enquadrada como Professora Nível Superior P2, 30 horas a partir de 1º de fevereiro de 2001. Após a servidora comprovar a formação superior para o exercício do cargo.
3. A servidora requereu a aposentadoria em 27 de novembro de 2018, fl. 42, pelo exercício do magistério na Escola Dom Julio Mattioli, no município de Sena Madureira, tendo sido efetuada a concessão de aposentadoria através da Portaria nº 79 de 26/01/2023 do Instituto de Previdência do Estado do Acre, publicada no DOE nº 13.464, de 30/01/2023;
4. O tempo de serviço líquido, apurado para fins da aposentadoria especial do magistério foi de 30 anos e 259 dias de contribuição, atendendo os requisitos para aposentadoria na modalidade voluntária integral especial por tempo de contribuição, nos termos do art. 4º da Emenda Constitucional nº 52, de 2 de dezembro de 2019, com proventos integrais de acordo com o § 6º, inciso I, do mesmo artigo.
5. A 4º IGCE deste Tribunal afirma, às fls. 203/204, que a servidora preenche todos os requisitos constitucionais para fazer jus à aposentadoria com enquadramento da fixação dos proventos em P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria estadual de Educação, com benefício iniciado em 30/01/2023, no valor de R$ 6.215,78 (seis mil duzentos e quinze reais e setenta e oito centavos)¹, conforme ao enquadramento final da carreira, mencionado no Ato de Fixação dos Proventos à fl. 21;
6. O Ministério Público de Contas pronunciou-se à fl. 262, através da sua Ilustre Procuradora, Dra. Anna Helena de Azevedo Lima.

É o Relatório.

Voto:
A EXMA. SENHORA CONSELHEIRA NALUH MARIA LIMA GOUVEIA (Relatora):

1. Trata-se de apreciação da legalidade e registro da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula nº 266027-1, no cargo de Professora de Nível Superior, 30 horas, Classe II, Referência J.
2. Posto isso, e tudo o que mais consta dos autos, e considerando a regularidade da documentação apresentada, assim como sua conformidade com a legislação vigente pertinente, inclusive levando em conta as análises do corpo técnico e do Ministério Público de Contas, e com fundamento no art. 61, inciso III, da Constituição Estadual de 1989, VOTO:

1) Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula n. 229962-1, no cargo de Professora, P2-30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 79, de 26 de janeiro de 2023, na modalidade aposentadoria voluntária Especial por tempo de contribuição, nos termos art. 4º, da Emenda Constitucional nº 52, de 2 de dezembro de 2019, com proventos integrais, de acordo com o § 6º, inciso I, do mesmo artigo, especificados no Ato de Fixação à fl. 21;
2) Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão, e
3) Após as formalidades de estilo, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, acordam os Membros da 1ª Câmara do Tribunal de Contas do Estado do Acre na 119ª Sessão Ordinária Virtual, por unanimidade, nos termos do voto da Conselheira-Relatora, em:

1. Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Maria do Perpétuo Socorro Diniz Andrade, matrícula n. 266027-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 79, de 26 de janeiro de 2023, na modalidade aposentadoria voluntária Especial por tempo de contribuição, nos termos art. 4º, da Emenda Constitucional nº 52, de 2 de dezembro de 2019, com proventos integrais, de acordo com o § 6º, inciso I, do mesmo artigo, especificados no Ato de Fixação à fl. 21;
2. Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre- ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

In [6]:
example4 = """
Relatório:
1. Trata-se de apreciação da legalidade e registro da aposentadoria voluntária por tempo de contribuição da SR.ª LUCÉLIA NAGÃO GONÇALVES, matriculada na Secretaria de Estado de Educação e Esporte sob o número 9130438-3, com proventos calculados pela média1, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, consoante se vê pela Portaria n. 863, de 07 de julho de 2017 (DOE n. 12.092, de 11-07-2017) - fls. 72/73, apresentada nesta Corte em 13-08-20192.

2. Quanto aos proventos, foram fixados em R$ 3.423,40 (três mil quatrocentos e vinte e três reais e quarenta centavos), no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, nos termos do Ato de Fixação dos Proventos à fl. 683, de acordo com o previsto no Anexo VII, da Lei Complementar Estadual n. 67, de 29-06-1999, modificado pela LCE n. 330, de 06 de março de 20174.

3. Após o registro, a autuação e a distribuição, os autos foram enviados à DIRETORIA DE AUDITORIA FINANCEIRA E ORÇAMENTÁRIA, que se manifestou por meio da 4ª INSPETORIA GERAL DE CONTROLE EXTERNO, pela admissibilidade da concessão de aposentadoria, em razão do preenchimento dos requisitos legais pertinentes à matéria - fls. 80/83.

4. O MINISTÉRIO PÚBLICO DE CONTAS, por meio de seu i. Procurador, Dr. João Izidro de Melo Neto, pronunciou-se à fl. 120.

5. É o breve Relatório.

Voto:
A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO DE ARAÚJO (Relatora):

1. Trata-se de apreciação da legalidade e registro da concessão de aposentadoria à SR.ª LUCÉLIA NAGÃO GONÇALVES, com proventos calculados pela média, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”.
2. Observa-se que após a edição da LCE n. 330/2017, que modificou o § 8º do artigo 29 da LCE n. 67/1999, foi determinado o reenquadramento dos servidores, observando-se o interstício de trinta e três meses para cada progressão.
3. Ante todo o exposto, e tudo o que mais consta dos autos, bem como considerando a regularidade da documentação apresentada e a conformidade do ato de concessão com o estabelecido na legislação em vigor, inclusive levando-se em conta as manifestações do corpo técnico e do Ministério Público de Contas, VOTO, com base no artigo 61, inciso III, da Constituição Estadual de 1989, pelo:
   3.1 reconhecimento da LEGALIDADE e, consequente, REGISTRO neste Tribunal de Contas, da aposentadoria voluntária por tempo de contribuição da SR.ª LUCÉLIA NAGÃO GONÇALVES, matrícula n. 9130438-3, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, do quadro de pessoal do Estado da Secretaria de Educação e Esporte, nos termos do artigo 40, § 1º, inciso III, alínea “a”, da Constituição Federal, com a redação dada pela Emenda Constitucional n° 41, de 19 de dezembro de 20035, com proventos fixados em R$ 3.423,40 (três mil quatrocentos e vinte e três reais e quarenta centavos), nos termos do Ato de Fixação dos Proventos à fl. 68, de acordo com o previsto no Anexo VII, da Lei Complementar Estadual n. 67, de 29-06-1999, modificado pela LCE n. 330, de 06 de março de 2017;
   3.2. ENCAMINHAMENTO de cópia do processo ao Instituto de Previdência do Estado do Acre, para proceder à ciência da interessada do teor da decisão, bem como para que observe o artigo 11, da Resolução-TCE n. 103, de 14-09-2016; e
   3.3. finalmente, após as formalidades de estilo, pelo ARQUIVAMENTO dos autos.

4. É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, ACORDAM os Membros da 2ª Câmara do Tribunal de Contas do Estado do Acre, na 127ª Sessão Ordinária Virtual, POR UNANIMIDADE, nos termos do voto da Conselheira-Relatora, em:

1. RECONHECER a LEGALIDADE e, consequentemente, REGISTRAR neste Tribunal de Contas, a aposentadoria voluntária por tempo de contribuição da SR.ª LUCÉLIA NAGÃO GONÇALVES, matrícula n. 9130438-3, com proventos calculados pela média, concedida por meio da Portaria n. 863, de 07 de julho de 2017, com fundamento no artigo 40, § 1º, inciso III, alínea “a”, da Constituição Federal, com a redação dada pela Emenda Constitucional n° 41, de 19 de dezembro de 2003, no cargo de PROFESSORA DE NÍVEL SUPERIOR, 30 HORAS, CLASSE II, REFERÊNCIA “F”, do quadro de pessoal do Estado da Secretaria de Educação e Esporte, com proventos fixados em R$ 3.423,40 (três mil quatrocentos e vinte e três reais e quarenta centavos), nos termos do Ato de Fixação dos Proventos à fl. 68, de acordo com o previsto no Anexo VII, da Lei Complementar Estadual n. 67, de 29-06-1999, modificado pela LCE n. 330, de 06 de março de 2017;
2. ENCAMINHAR cópia do processo ao Instituto de Previdência do Estado do Acre, para proceder à ciência da interessada do teor da decisão, bem como para que observe o artigo 11, da Resolução-TCE n. 103, de 14-09-2016; e c) ARQUIVAR os autos, após as formalidades de estilo.
"""

In [7]:
example5 = """
Relatório:
1. Tratam os autos da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra Arinete Oliveira Souza Barros, matrícula nº 244503-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J.
2. A servidora foi admitida na Secretaria Estadual de Educação por concurso público em 22 de maio de 1992 para ocupar o cargo de Professora PS2. Posteriormente foi enquadrada no cargo P1, 25 horas, por meio da Portaria n. 935, de 25/01/2002, recebendo todas as promoções e progressões da carreira do magistério, nos termos da Lei complementar n. 67/1999.
3. Com base na Lei Complementar n. 213, de 18/06/2010, obteve progressão funcional para o cargo de professora de Nível Superior 30 horas, a partir de outubro daquele mesmo ano. Após a servidora comprovar a formação em nível superior em pedagogia, concluindo no ano de 2010 e especialização em 2011.
4. A servidora requereu a aposentadoria em 2 de janeiro de 2018, tendo sido efetuada a concessão de aposentadoria através da Portaria nº 242 de 02/04/2018 do Instituto de Previdência do Estado do Acre, publicada no DOE nº 12.272 de 03/04/2018;
5. O tempo de serviço líquido, apurado para fins da aposentadoria especial do magistério foi de 26 anos e 221 dias de contribuição, atendendo os requisitos para aposentadoria na modalidade voluntária integral especial por tempo de contribuição, nos termos do art. 6 º, incisos I, II, III e IV, da Emenda Constitucional nº 41, de 19 de dezembro de 2003, c/c art. 2º, da Emenda Constitucional n.º 47, de 05 de julho de 2005, e art. 95 da Lei Complementar Estadual n. 154/2005;

6. A 4º IGCE deste Tribunal afirma, às fls. 62/63 que a servidora preenche todos os requisitos constitucionais para fazer jus à aposentadoria com enquadramento da fixação dos proventos em P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria estadual de Educação, com benefício iniciado em 03/04/2018, no valor de R$ 5.725,39 (cinco mil setecentos e vinte e cinco reais e trinta e nove centavos)1, conforme ao enquadramento final da carreira, mencionado no Ato de Fixação dos Proventos à fl. 59, e

7. O Ministério Público de Contas pronunciou-se à fl. 122, através do seu Ilustre Procurador, Dr. Sérgio Cunha Mendonça.

É o Relatório.

Voto:
A EXMA. SENHORA CONSELHEIRA NALUH MARIA LIMA GOUVEIA (Relatora):

1. Trata-se de apreciação da legalidade e registro da aposentadoria da servidora pública da Secretaria de Educação do Estado do Acre, Sra. Arinete Oliveira Souza Barros, matrícula nº 244503-1, no cargo de Professora de Nível Superior, 30 horas, Classe II, Referência J.
2. Posto isso, e tudo o que mais consta dos autos, e considerando a regularidade da documentação apresentada, assim como sua conformidade com a legislação vigente pertinente, inclusive levando em conta as análises do corpo técnico e do Ministério Público de Contas, e com fundamento no art. 61, inciso III, da Constituição Estadual de 1989, VOTO:

1) Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Arinete Oliveira Souza Barros, matrícula n. 244503-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 242, de 2 de abril de 2018, na modalidade aposentadoria voluntaria Especial por tempo de contribuição, nos termos do art. 6º, incisos I, II, III e IV, da Emenda Constitucional nº 41, de 19 de dezembro de 2003, c/c art. 2º, da Emenda Constitucional n.º 47, de 05 de julho de 2005, e art. 95 da Lei Complementar Estadual n. 154/2005, com proventos estabelecidos no Ato de Fixação à fl. 59;

2) Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre - ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão, e

3) Após as formalidades de estilo, pelo arquivamento dos autos.

É como VOTO.

Acordão:
Vistos, relatados e discutidos os autos do processo acima identificado, acordam os Membros da 1ª Câmara do Tribunal de Contas do Estado do Acre na 119ª Sessão Ordinária Virtual, por unanimidade, nos termos do voto da Conselheira-Relatora, em:
1. Reconhecer a legalidade e, consequentemente, registrar neste Tribunal de Contas a aposentadoria da Sra. Arinete Oliveira Souza Barros, matrícula n. 244503-1, no cargo de Professora, P2- 30 horas, Classe II, Referência J, do quadro de pessoal da Secretaria Estadual de Educação do Acre, concedida por meio da Portaria n. 242, de 2 de abril de 2018, na modalidade aposentadoria voluntária Especial por tempo de contribuição, nos termos do art. 6º, incisos I, II, III e IV, da Emenda Constitucional nº 41, de 19 de dezembro de 2003, c/c art. 2º, da Emenda Constitucional n.º 47, de 05 de julho de 2005, e art. 95 da Lei Complementar Estadual n. 154/2005, com proventos estabelecidos no Ato de Fixação à fl. 59;
2. Encaminhar cópia do Acórdão proferido ao Instituto de Previdência do Estado do Acre- ACREPREVIDÊNCIA, para que tome as providências necessárias, inclusive dando ciência à interessada do teor da decisão. Após as formalidades de estilo, pelo arquivamento dos autos.
"""

### Promps Definition

In [8]:
base_prompt = """
Você é um assistente jurídico especializado na redação de acórdãos do Tribunal de Contas do Estado do Acre. A partir das informações fornecidas, você deve elaborar um acórdão completo que siga uma estrutura clara, coesa e que inclua os seguintes elementos em um texto corrido:

1. **Cabeçalho**: Inicie o acórdão com o nome do tribunal (Tribunal de Contas do Estado do Acre), o número do processo e a data de julgamento.
2. **Relator e Membros do Órgão Julgador**: Identifique o relator do processo, bem como os demais membros do órgão colegiado (câmara, turma, seção, órgão especial, plenário etc.) que participaram do julgamento. Informe também o resultado da votação (unânime ou com voto vencido).
3. **Resumo dos Fatos**: Resuma os fatos principais do processo, incluindo as alegações, os documentos relevantes e os argumentos apresentados pelas partes envolvidas.
4. **Fundamentação**: Apresente os fundamentos jurídicos da decisão. Isso deve incluir a análise das questões de direito envolvidas, as normas aplicáveis e a interpretação dos membros do órgão colegiado.
5. **Decisão**: Expresse de forma clara e objetiva a decisão do colegiado, indicando os resultados do julgamento e as providências a serem tomadas, caso aplicável (por exemplo, aprovação ou reprovação de contas, determinação de devolução de recursos, etc.).
6. **Voto Vencido** (se houver): Caso a votação não seja unânime, transcreva o voto vencido de um dos membros, expondo sua argumentação divergente.\n\n
"""

In [9]:
zero_shot_prompt = base_prompt + """
Com base no relatório e no voto abaixo, redija o texto do acordão:

Relatório:
{relatorio}

Voto:
{voto}

Acordão:
"""

In [10]:
one_shot_prompt = base_prompt + """
Exemplo:
{example}

A partir do exemplo acima, 
""" + zero_shot_prompt

In [11]:
def generate_few_shot_prompt(examples):
    prompt = base_prompt
    for i in range(len(examples)):
        prompt += f"""
        Exemplo {i + 1}:
        {examples[i]}\n\n
        """
    return """
        A partir dos exemplos acima, 
        """ + zero_shot_prompt

### Experiment

In [ ]:
keys = ['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', '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', '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', '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', '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', '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', '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', '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', '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']

In [12]:
import concurrent.futures
import time

def invoke_model(model, prompt):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(model.invoke, prompt)
        try:
            return future.result(timeout=60)
        except concurrent.futures.TimeoutError:
            raise TimeoutError("A execução excedeu o tempo limite.")

def generate(model, prompt, temperature, use_ollama = True):
    if use_ollama:
        client = ChatOllama(
            model=model,
            temperature=temperature,
        )
        model = (client | StrOutputParser())
        for attempt in range(3):
            try:
                return invoke_model(model, prompt)
            except TimeoutError:
                print(f"Timeout na tentativa {attempt + 1}. Retentando...")
            except Exception as e:
                print(f"Erro ao invocar o modelo: {e}")
                break
    else:
        random.shuffle(keys)
        for key in keys:
            try:
                client = Groq(api_key=key)
                response = client.chat.completions.create (
                    messages=[{"role": "user", "content": prompt}],
                    model=model,
                    temperature=temperature,
                )
                return response.choices[0].message.content
            except:
                print("key expirada")

In [13]:
examples = [example1, example2, example3, example4, example5]
models = ["llama3.1"]
temperatures = [0.25, 0.5, 0.75, 1.0]
# n_exemples = [0, 1, 2, 3, 4, 5]
n_exemples = [1, 2]

In [14]:
for n_example, model, temp in itertools.product(n_exemples, models, temperatures):
    results = []
    filename = f'../results/llama-3.1-8b-instant-{n_example}-examples-temperature-{temp}.csv'
    if not os.path.isfile(filename):
        with tqdm.tqdm(total=len(df), smoothing=0.1) as pbar:
            for idx, row in df.iterrows():
                relatorio = row["relatorio"]
                voto = row["voto"]
                prompt = ""
                if n_example == 0:
                    prompt = zero_shot_prompt.format(
                        relatorio=relatorio,
                        voto=voto,
                    )
                elif n_example == 1:
                    prompt = one_shot_prompt.format(
                        relatorio=relatorio,
                        voto=voto,
                        example=examples[0]
                    )
                else:
                    prompt = generate_few_shot_prompt(examples[:n_example]).format(
                        relatorio=relatorio,
                        voto=voto,
                    )

                response = ""
                while True:
                    try:
                        response = generate(model, prompt, temp)
                        break
                    except Exception as e:
                        print('Error: ' + str(e) + ' (Retrying)')
                        time.sleep(10)

                results.append({
                    "relatorio": relatorio,
                    "voto": voto,
                    "acordao": row['acordao'],
                    "generated": response,
                    "model": model,
                    'n_examples': n_example,
                    "temperature": temp
                })

                pd.DataFrame(results).to_csv(filename, index=False)
                pbar.update(1)

  1%|▏         | 2/150 [00:15<20:01,  8.12s/it]

Timeout na tentativa 1. Retentando...


 88%|████████▊ | 132/150 [36:17<04:35, 15.33s/it]

Timeout na tentativa 1. Retentando...


100%|██████████| 150/150 [44:57<00:00, 17.98s/it]


# MÉTRICAS

In [15]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from nltk import word_tokenize
import nltk
from bert_score import BERTScorer
import torch.nn.functional as F
import evaluate
import os

c:\Users\enied\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
nltk.download('wordnet')
nltk.download('punkt_tab')

scorer_bert = BERTScorer(model_type='bert-base-multilingual-cased')
bleu = evaluate.load('bleu')

def calculate_rouge_score(generated, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference, generated)

def calculate_bleu_score(generated, reference):
    score = bleu.compute(predictions=[generated], references=[[reference]])
    return score['bleu']

def calculate_meteor_score(generated, reference):
    tokenized_reference = word_tokenize(reference)
    tokenized_generated = word_tokenize(generated)
    score = meteor_score.meteor_score([tokenized_reference], tokenized_generated)
    return score

def calculate_similarity(embedding1, embedding2):
    similarity_matrix = F.cosine_similarity(embedding1.unsqueeze(1), embedding2.unsqueeze(0), dim=-1)
    return similarity_matrix

def calculate_bert_score(generated, reference):
    return scorer_bert.score([generated], [reference])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\enied\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\enied\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [19]:
calc_result_df = pd.read_csv('../evaluation.csv')

In [ ]:
files_path = '../results/'

dfs = []
for file in os.listdir(files_path):
    if file.endswith('.csv'):
        df_file = pd.read_csv(os.path.join(files_path, file))
        dfs.append(df_file)

result_df = pd.concat(dfs, ignore_index=True)
result_df.sample(10)

,relatorio,voto,acordao,generated,model,n_examples,temperature
0,Tratam os autos da aposentadoria da servidora ...,Trata-se de apreciação da legalidade e registr...,"Vistos, relatados e discutidos os autos do pro...",Parece haver um erro no número da matrícula no...,llama3.1,1,1.0
1,1. Tratam os autos da Prestação de Contas do F...,A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...,"Vistos, relatados e discutidos os autos do pro...",Com base na análise da prestação de contas do ...,llama3.1,1,1.0
2,1. Tratam os autos da Prestação de Contas da S...,A EXMA. SENHORA CONSELHEIRA DULCINÉA BENÍCIO D...,"Vistos, relatados e discutidos os autos do pro...","Sim, posso ajudar a identificar os pontos rele...",llama3.1,1,1.0
3,"1. Trata-se os autos de Denúncia, oriunda do o...",A EXMA. SENHORA CONSELHEIRA NALUH MARIA LIMA G...,"Vistos, relatados e discutidos os autos do pro...",A Corte de Contas do Estado do Acre decidiu nã...,llama3.1,1,1.0
4,1. Trata o presente processo da aposentadoria ...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...,"Vistos, relatados e discutidos os autos do pro...",Não foi possível extrair informações relevante...,llama3.1,1,1.0
...,...,...,...,...,...,...,...
745,1. Tratam os autos da Prestação de Contas do G...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...,"Vistos, relatados e discutidos os autos do pro...",**Tribunal de Contas do Estado do Acre**\n\n**...,llama3.1,2,1.0
746,1. Tratam os autos de Denúncia protocolada na ...,O EXMO. SENHOR CONSELHEIRO ANTONIO CRISTOVÃO C...,"Vistos, relatados e discutidos os autos do pro...",Tal acórdão não contém informações de natureza...,llama3.1,2,1.0
747,Tratam os autos da aposentadoria da servidora ...,Trata-se de apreciação da legalidade e registr...,"Vistos, relatados e discutidos os autos do pro...",**Tribunal de Contas do Estado do Acre**\n\n**...,llama3.1,2,1.0
748,1. Trata o presente processo da aposentadoria ...,A EXMA. SENHORA CONSELHEIRA-SUBSTITUTA MARIA D...,"Vistos, relatados e discutidos os autos do pro...",Tribunal de Contas do Estado do Acre.\n\nCódig...,llama3.1,2,1.0


In [23]:
result_df[result_df.isna().any(axis=1)].groupby(['model', 'n_examples', 'temperature'])['generated'].apply(len)

Series([], Name: generated, dtype: object)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

def evaluate_result(df_result):
    with tqdm.tqdm(total= len(df_result), smoothing=0.2) as pbar:
        for i, row in df_result.iterrows():
            bert_score_precision, bert_score_recall, bert_score_f1 = calculate_bert_score(row['generated'], row['acordao'])
            bleu_score = calculate_bleu_score(row['generated'], row['acordao'])
            meteor_score = calculate_meteor_score(row['generated'], row['acordao'])
            rouge_result = calculate_rouge_score(row['generated'], row['acordao'])
            rouge1 = rouge_result['rouge1'][2]
            rouge2 = rouge_result['rouge2'][2]
            rougeL = rouge_result['rougeL'][2]
            
            df_result.loc[i, 'bert_score_precision'] = float(bert_score_precision)
            df_result.loc[i, 'bert_score_recall'] = float(bert_score_recall)
            df_result.loc[i, 'bert_score_f1'] = float(bert_score_f1)
        
            df_result.loc[i, 'bleu_score'] = bleu_score
            df_result.loc[i, 'meteor_score'] = meteor_score
            
            df_result.loc[i, 'rouge1'] = rouge1
            df_result.loc[i, 'rouge2'] = rouge2
            df_result.loc[i, 'rougeL'] = rougeL

            pbar.update(1)
    df_result.to_csv('../evaluation.csv', index=False)

In [ ]:
evaluate_result(result_df)

100%|██████████| 750/750 [10:01<00:00,  1.25it/s]
